# Forward Propagation Code for a Sphere


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [18]:
def SphericalThickness(r,x,y):
    '''
    Returns thickness through sphere of radius r at a given (x,y)
    '''
    if x**2 + y**2 < r**2:
        return np.sqrt(r**2 - (x**2 + y**2)) * 2
    else:
        return 0.


radius = 50e-3   # m
d = 250e-3       # m
Lx = 100e-3      # m
Ly = 100e-3      # m
nbins_x = 101
nbins_y = 101

h = 6.626e-34    # J.s
c = 2.99e8       # m/s
E = 9.61306e-13  # J
delta = 1.2743e-5
beta = 3.8559e-8
n = 1 - delta + 1j*beta
k = ( (2*np.pi)/(h*c) ) * E * n

attenuated_wave = np.empty((nbins_x,nbins_y), dtype = np.complex128)

for i in range(nbins_x):
    x = (i - nbins_x/2) / Lx
    for j in range(nbins_y):
        y = (j - nbins_y/2) / Ly
        t = SphericalThickness(radius,x,y)
        attenuated_wave[i,j] = np.exp(1j * k * radius) * np.exp(-k*t)

x = np.array([i - nbins_x/2 for i in range(nbins_x)])
y = np.array([j - nbins_y/2 for j in range(nbins_y)])

print(attenuated_wave)

# plt.hist2d(X,Y,np.abs(attenuated_wave))
# plt.show()

[[ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 ..., 
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...,  0.+0.j  0.+0.j  0.+0.j]]


In [ ]:
def SphericalAttenuation_pw(r, nbins_x, nbins_y):
    '''
    Inputs:
        r (float): radius
    Outputs:
        attenuated_wave (ndarray): xy grid of waveform values post-attenuation
    '''
    grid = np.zeros((nbins_x,nbins_y))
    
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            makesphere.exe
            kthxbai
